# 1. Backpropagation

Okay, let's get started. I'm doing this because a year into my career as a machine learning engineer, I was asked to code backpropagation for a simple neural network from scratch and failed miserably. So I thought it might be good to dive back into the roots and really understand what's going on, before I go back to using transformers and convolutions and SwiGLUs and whatever else is going on in the industry these days. If you're like me and got into all this without any solid educational foundation in deep learning from a prestigious institution, then feel free to follow along. But enough about me. We're here to do some math and coding. Hopefully not too much math.

## 1.1 Math

Ah dang. Well, whatever. The intuition of backpropagation is quite simple. We have a function $f(x) = y$ and we'd like to minimize the loss function $L(y)$. Consider a simple example where $f(x)$ is a neural network with 2 linear layers, with the sigmoid activation after each one:

$f(x) = S(L_2(S(L_1(x)))) = S((S(XW_1 + b_1))W_2 + b_2))$

To update the parameters $W_1, W_2, b_1$, and $b_2$, we need to calculate the loss gradient with respect to each of them, which is the current rate of change of the loss with respect to each specific parameter. Moving each parameter in the opposite direction of these gradients will cause the loss to decrease.

We can obtain these gradients by taking the original loss gradient with respect to the output and backpropagating it through the layers, multiplying by each component as per the chain rule to get the value we need. For example, if we want the loss gradient with respect to $b_2$, we must obtain:

$\frac{dL}{db_2} = \frac{dL}{dS(L_2(S(L_1(x))))} * \frac{dS(L_2(S(L_1(x))))}{dL_2(S(L_1(x)))} * \frac{dL_2(S(L_1(x)))}{db_2}$

Essentially, we need to use the chain rule to unravel the neural network function until we have $L_2(S(L_1(x))) = (S(XW_1 + b_1))W_2 + b_2$, the part of the function that directly involves $b_2$.

Similarly, if we want it with respect to $b_1$, the formula is:

$\frac{dL}{db_1} = \frac{dL}{dS(L_2(S(L_1(x))))} * \frac{dS(L_2(S(L_1(x))))}{dL_2(S(L_1(x)))} * \frac{dL_2(S(L_1(x)))}{dS(L_1(x))} * \frac{dS(L_1(x))}{dL_1(x)} * \frac{dL_1(x)}{db_1}$

As you can see, the two gradients share some computation. Moreover, the expression $\frac{dL_2(S(L_1(x)))}{dS(L_1(x))}$ in the equation above can only be calculated using the inputs and outputs of layer 2: thus for layer 1 to be able to obtain the gradients for $W_1$ and $b_1$, it must receive a gradient from layer 2 that has already processed some of the computation, namely:

$grad = \frac{dL}{dS(L_2(S(L_1(x))))} * \frac{dS(L_2(S(L_1(x))))}{dL_2(S(L_1(x)))} * \frac{dL_2(S(L_1(x)))}{dS(L_1(x))}$

The gradients for layer 1 can then be obtained by simply multiplying on top of it like so:

$\frac{dL}{db_1} = grad * \frac{dS(L_1(x))}{dL_1(x)} * \frac{dL_1(x)}{db_1}$

As such, the loss gradient must be propagated backwards through all the layers, each one performing the computation required for the next layer back.

## 1.2 Implementation

Basically, for each component of the forward pass, we need to implement a backward pass that "undoes" the function: performing the gradient computation for that component and multiplying it by the current gradient being propagated, exposing the inner function containing the parameters we care about in the next layer.

At the very top of the chain, we have the loss gradient with respect to the output $\frac{dL}{dy}$, where $y = S(L_2(S(L_1(x))))$.

$grad = \frac{dL}{dy}$

We can then go one function down, to the last activation function. We have no parameters to update here, so we just obtain the gradient we need to pass backwards.

$grad = grad * \frac{dS(L_2(S(L_1(x))))}{dL_2(S(L_1(x)))} $

Let's write the backward pass for the sigmoid activation:

In [ ]:
import numpy as np

class Sigmoid:
    def forward(self, x):
        self.x = x
        return 1 / (1 + np.exp(-x))

    def backward(self, grad):
        # dx = σ(x) * (1 - σ(x))
        return grad * Sigmoid(self.x) * (np.ones(Sigmoid(self.x).shape) - Sigmoid(self.x))

Pretty simple. We cache the input $x$ during the forward pass, then during the backwards pass, we get the propagated gradient, multiply it by the derivative of the sigmoid function with respect to the input, and pass it further down. If you'd like to see how to get the derivative of the sigmoid function, you can just look it up, because I don't really want to add it here. Sorry.

The next function down is the linear layer, where we have to update the weights and bias. We also need to get the gradient to pass backwards.

$\frac{dL}{dW_2} = grad * \frac{dL_2(S(L_1(x)))}{dW_2}$

$\frac{dL}{db_2} = grad * \frac{dL_2(S(L_1(x)))}{db_2}$

$grad = grad * \frac{dL_2(S(L_1(x)))}{dS(L_1(x))}$

So how do we implement this?

Consider a linear layer with output dimension 1, which you'd commonly find at the end of a regression network. The output $y$ of this layer is:

$y = XW + b = \begin{pmatrix}x_1 & x_2 & ... & x_n\end{pmatrix}\begin{pmatrix}w_1 \\ w_2 \\ ... \\ w_n\end{pmatrix} + b = x_1 w_1 + x_2 w_2 + ... + x_n w_n + b$

We want to obtain the values $\frac{dy}{dW}$ and $\frac{dy}{db}$, so we can multiply it by the gradient being propagated to receive our desired gradients $\frac{dL}{dW}$ and $\frac{dL}{db}$.

$\frac{dy}{dW} = \begin{pmatrix}\frac{dy}{dw_1} \\ \frac{dy}{dw_2} \\ ... \\ \frac{dy}{dw_n}\end{pmatrix} = \begin{pmatrix}x_1 \\ x_2 \\ ... \\ x_n\end{pmatrix} = X^T$

$\frac{dy}{db} = 1$

Because $y$ is a single scalar value, we have:

$\frac{dL}{dW} = \frac{dL}{dy} * \frac{dy}{dW} = \frac{dL}{dy} * X^T$

$\frac{dL}{db} = \frac{dL}{dy} * \frac{dy}{db} = \frac{dL}{dy} * 1 = \frac{dL}{dy}$

And to obtain the gradient to pass further backwards (see if you can figure this one out yourself!):

$\frac{dL}{dx} = \frac{dL}{dy} * \frac{dy}{dx} = \frac{dL}{dy} * W^T$

Now we can implement most of the linear layer:

In [ ]:
class Linear:
    def __init__(self, input_dim, output_dim):
        self.w = np.random.normal(0, 1, [input_dim, output_dim])
        self.b = np.random.normal(0, 1, [output_dim])

    def forward(self, x):
        self.x = x
        x = x @ self.w
        x = x + self.b
        return x

    def backward(self, grad):
        
        # dL_dy = grad
        # dL_dw = dL_dy @ self.x.T
        # dL_db = dL_dy
        # dL_dx = dL_dy @ self.w.T

        # TODO: gradient descent here

        # return dL_dx
        return

But unfortunately, just like your first couple attempts at a LeetCode medium, this implementation will fail catastrophically when integrated into a full neural network. You will get a ton of shape mismatch errors, and... actually, that's it. Just shape mismatch errors.

And to solve that, we need to account for two very common cases:

1. What happens when the output $y$ is not a scalar quantity?
2. What if we want to process multiple samples in a single pass?

And now we have an entire other section. Great.

## 1.3 Hidden Layers and Batch Dimension

Consider a hidden layer where the output dimension $>1$. $\frac{dL}{dy}$ is now a vector quantity $\begin{pmatrix}\frac{dL}{dy_1} & \frac{dL}{dy_2} &  ... & \frac{dL}{dy_n}\end{pmatrix}$